In [1]:
!pip install -q kaggle

In [2]:
import os
import random
import numpy as np
from abc import ABC, abstractmethod

from sklearn.metrics import f1_score

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader

from PIL import Image
from tqdm import tqdm

from google.colab import files

#Download

- If you want run this code, you need to create the Kaggle API key. The next step was extracted  [here](https://www.kaggle.com/discussions/general/74235)

In [ ]:
files.upload()

In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json  ~/.kaggle/
!chmod 600 kaggle.json
!kaggle datasets download -d arpitjain007/dog-vs-cat-fastai
!unzip dog-vs-cat-fastai.zip
!mv dogscats/valid dogscats/test

#Load and proccess data

In [5]:
#test
dog_test_path = "dogscats/test/dogs"
cat_test_path = "dogscats/test/cats"

dog_test = os.listdir(dog_test_path)
cat_test = os.listdir(cat_test_path)

dog_test_paths = [os.path.join(dog_test_path, filename) for filename in dog_test]
cat_test_paths = [os.path.join(cat_test_path, filename) for filename in cat_test]

test_data = dog_test_paths + cat_test_paths
random.shuffle(test_data)

#train y val
dog_train_path = "dogscats/train/dogs"
cat_train_path = "dogscats/train/cats"

dog_train = os.listdir(dog_train_path)
cat_train = os.listdir(cat_train_path)

dog_train_paths = [os.path.join(dog_train_path, filename) for filename in dog_train]
cat_train_paths = [os.path.join(cat_train_path, filename) for filename in cat_train]

split_ratio = 0.8
num_dog_train = int(len(dog_train_paths) * split_ratio)
num_cat_train = int(len(cat_train_paths) * split_ratio)

dog_train_data = dog_train_paths[:num_dog_train]
cat_train_data = cat_train_paths[:num_cat_train]

dog_val_data = dog_train_paths[num_dog_train:]
cat_val_data = cat_train_paths[num_cat_train:]

train_data = dog_train_data + cat_train_data
val_data = dog_val_data + cat_val_data

random.shuffle(train_data)
random.shuffle(val_data)

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
class CustomDataset(Dataset):
    """
    Custom dataset class for loading images from file paths and applying transformations.

    Args:
        file_paths (list): List of file paths for the images.
        transform (callable, optional): A function/transform that takes in an image and returns a
            transformed version. Default is None.

    Methods:
        __init__(self, file_paths, transform=None):
            Initializes a new instance of the CustomDataset class.

        __len__(self):
            Returns the total number of samples in the dataset.

        __getitem__(self, idx):
            Retrieves the sample at the specified index.

    Attributes:
        file_paths (list): List of file paths for the images.
        transform (callable): A function/transform applied to the image.

    Example:
        transform = transforms.Compose([
            transforms.Resize((64, 64)),
            transforms.ToTensor(),
        ])

        # Example usage:
        dataset = CustomDataset(file_paths=train_data, transform=transform)
    """
    def __init__(self, file_paths, transform=None):
        self.file_paths = file_paths
        self.transform = transform

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        img_path = self.file_paths[idx]
        image = Image.open(img_path).convert('RGB')
        label = 1 if 'dog.' in img_path else 0

        if self.transform:
            image = self.transform(image)

        return image, label

transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
])

train_dataset = CustomDataset(train_data, transform=transform)
val_dataset = CustomDataset(val_data, transform=transform)
test_dataset = CustomDataset(test_data, transform=transform)

# Build Network

In [8]:
class EvolvedCNN(nn.Module):
    """
      EvolvedCNN is a convolutional neural network with an evolved architecture.

      Args:
          hyperparameters (list): List of hyperparameters representing the number of neurons
                                  in each convolutional layer.
          image_size (tuple): Tuple representing the input image size (height, width).

      Attributes:
          conv_layer_1 (nn.Sequential): First convolutional block.
          conv_layer_2 (nn.Sequential): Second convolutional block.
          conv_layer_3 (nn.Sequential): Third convolutional block.
          conv_layer_4 (nn.Sequential): Fourth convolutional block.
          image_size (tuple): Input image size.
          classifier (nn.Sequential): Classifier block with a linear layer.

      Methods:
          build_conv_block(in_channels, out_channels): Helper method to build a convolutional block.

      Forward:
        Perform forward pass through the convolutional blocks and classifier.
    """
    def __init__(self, hyperparameters, image_size):
        super(EvolvedCNN, self).__init__()
        self.conv_layer_1 = self.build_conv_block(3, hyperparameters[0])
        self.conv_layer_2 = self.build_conv_block(hyperparameters[0], hyperparameters[1])
        self.conv_layer_3 = self.build_conv_block(hyperparameters[1], hyperparameters[2])
        self.conv_layer_4 = self.build_conv_block(hyperparameters[2], hyperparameters[3])

        self.image_size = image_size
        in_features = hyperparameters[3] * 4 * 4

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features, out_features=2))

    def build_conv_block(self, in_channels, out_channels):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(out_channels),
            nn.MaxPool2d(2)
        )

    def forward(self, x):
        x = self.conv_layer_1(x)
        x = self.conv_layer_2(x)
        x = self.conv_layer_3(x)
        x = self.conv_layer_4(x)
        x = self.classifier(x)
        return x


#Train phase

In [9]:
def train(train_loader, model, criterion, optimizer):
    """
    Train the model on a training dataset.

    Args:
        train_loader (DataLoader): DataLoader for the training dataset.
        model (nn.Module): Model to train.
        criterion (nn.Module): Loss function.
        optimizer (torch.optim.Optimizer): Optimizer.
        epoch (int): Number of epochs.

    Returns:
        float: Average loss on the training dataset.
    """
    model.train()
    total_loss = 0.0


    for images, labels in tqdm(train_loader, desc=f'Train step', leave=False):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    average_loss = total_loss / len(train_loader)
    return average_loss

def validation(test_loader, model, criterion):
    """
    Evaluate the model on a test dataset.

    Args:
        test_loader (DataLoader): DataLoader for the test dataset.
        model (nn.Module): Model to evaluate.
        criterion (nn.Module): Loss function.

    Returns:
        float: Average loss on the test dataset.
    """
    model.eval()
    total_loss = 0.0

    with torch.no_grad():
        for images, labels in tqdm(test_loader, desc="Evaluation step", leave=False):
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            total_loss += loss.item()

    average_loss = total_loss / len(test_loader)
    return average_loss


def evaluation(test_loader, model, criterion):
    """
    Evaluate the model on a test dataset.

    Args:
        test_loader (DataLoader): DataLoader for the test dataset.
        model (nn.Module): Model to evaluate.
        criterion (nn.Module): Loss function.

    Returns:
        float: Average F1-score on the test dataset.
    """
    model.eval()
    all_preds = []
    all_labels = []
    total_loss = 0.0

    with torch.no_grad():
        for images, labels in tqdm(test_loader, desc="Evaluation step", leave=False):
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            preds = torch.argmax(outputs, dim=1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    f1 = f1_score(all_labels, all_preds, average='weighted')
    average_loss = total_loss / len(test_loader)
    return average_loss, f1

def train_step(train_loader, val_loader, model, criterion, optimizer, epochs):
  """
    Train and validate a model over multiple epochs.

    Args:
        train_loader (DataLoader): DataLoader for the training dataset.
        val_loader (DataLoader): DataLoader for the validation dataset.
        model (nn.Module): Model to train and validate.
        criterion (nn.Module): Loss function.
        optimizer (torch.optim.Optimizer): Optimizer.
        epochs (int): Number of epochs.

    Returns:
        nn.Module: Trained model.
  """
  #loss_thr=10000
  for epoch in range(epochs):
      average_loss_train = train(train_loader, model, criterion, optimizer)
      print(f"average_loss_train de la época: {epoch} -- train_loss: {average_loss_train}")
      average_loss_val = validation(val_loader, model, criterion)
      print(f"average_loss_val de la época: {epoch} -- val_loss: {average_loss_val}")
      #if average_loss_val < loss_thr:
  return model

#Genetic algorithm

- Our chromosome is a list of candidate neurons to build a convolutional neural network. \[32, 64,128, 256, 512] (Surround 120 possible architectures)

- Fitness is the f1_score in the test step, after training our network in 5 epochs.

- Two points were used for the crossing, the parent selectors were Tournament without replacement and finally the mutation, exchange (same as N-Queens).

In [10]:
def init_pop(num_elements, neurons_candidates = [32, 64,128, 256, 512]):
  """
    Initialize a population of neural network architectures with random hyperparameters.

    Args:
        num_elements (int): Number of architectures to generate.
        neurons_candidates (list): List of possible values for the number of neurons.

    Returns:
        list: List of hyperparameter combinations representing neural network architectures.
  """
  i = 0
  hyperparameters_custom = []

  while i < num_elements:
    candidates = random.choices(neurons_candidates, k=4)
    if candidates not in hyperparameters_custom:
      hyperparameters_custom.append(candidates)
      i+=1
  return hyperparameters_custom

class Chromosome():
    """
        Initializes a Chromosome object.

        Args:
            chromosome (list): The genetic representation of the chromosome.
            fitness (float): The fitness score of the chromosome.
    """
    def __init__(self, chromosome, fitness):
        self.chromosome = chromosome
        self.fitness = fitness


def fitness(chromosome, train_loader, val_loader, test_loader, model, criterion, epochs=10, learning_rate=1e-4, image_size = (64, 64)):
    """
      Evaluate the performance of a chromosome in terms of F1-score on a test dataset.

      Args:
        chromosome (list): List of model hyperparameters.
        train_loader (DataLoader): DataLoader for the training dataset.
        val_loader (DataLoader): DataLoader for the validation dataset.
        test_loader (DataLoader): DataLoader for the test dataset.
        model (nn.Module): Evolved architecture model.
        criterion (nn.Module): Loss function.
        epochs (int): Number of epochs for training.
        learning_rate (float): Learning rate for the optimizer.
        image_size (tuple): Size of the input image.

      Returns:
        float: Average F1-score on the test dataset.
    """
    evolved_cnn = model(chromosome, image_size).to(device)
    optimizer = optim.Adam(evolved_cnn.parameters(), lr=learning_rate)
    evolved_cnn = train_step(train_loader, val_loader, evolved_cnn, criterion, optimizer, epochs)
    average_loss_test, f1_score_test = evaluation(test_loader, evolved_cnn, criterion)
    print(f"Test -> average_loss_test: {average_loss_test} -- Metric: f1 score: {f1_score_test}")
    return f1_score_test

def crossover_two_points(parent1, parent2, crossover_rate=0.7):
    """
    Perform two-point crossover between two chromosomes.

    Args:
        parent1 (list): First chromosome.
        parent2 (list): Second chromosome.

    Returns:
        tuple: Two descendants generated through two-point crossover.
    """
    if random.random() < crossover_rate:
        return parent1, parent2

    point1 = random.randint(0, len(parent1) - 1)
    point2 = random.randint(point1, len(parent1))

    child1 = parent1[:point1] + parent2[point1:point2] + parent1[point2:]
    child2 = parent2[:point1] + parent1[point1:point2] + parent2[point2:]

    return child1, child2


def mutation(chromosome, mutation_rate=0.5):
    """
    Apply mutation on a chromosome by performing a swap between two genes.

    Args:
        chromosome (list): Chromosome to mutate, represented as a list.
        mutation_rate (float): Mutation rate controlling the probability of applying the mutation.
                              It should be in the range [0, 1]. Default is 0.5.

    Returns:
        list: Mutated chromosome.
    """
    mutated_chromosome = chromosome.copy()

    if random.random() <= mutation_rate:
        index1, index2 = random.sample(range(len(chromosome)), 2)

        mutated_chromosome[index1], mutated_chromosome[index2] = mutated_chromosome[index2], mutated_chromosome[index1]

    return mutated_chromosome

def build_chromosome(pop, train_loader, val_loader, test_loader, model, criterion,\
                     epochs=10, learning_rate=1e-4, image_size=(64, 64)):
  chromosome = []
  print("build chromosomes step....")
  for config in pop:
        fitness_value = fitness(config, train_loader, val_loader, test_loader, model, criterion,\
                            epochs=epochs, learning_rate=learning_rate, image_size=image_size)
        chromosome.append(Chromosome(config, fitness_value))
  return chromosome

In [11]:
class Selection(ABC):
    """
    Clase abstracta para representar una selección en un algoritmo genético.

    Args:
        chromosomes (list): Lista de cromosomas.
        SOLUTIONS (numpy.ndarray): Espacio de soluciones.
    """
    def __init__(self, chromosomes):
        self.chromosomes = chromosomes

    @abstractmethod
    def get_indiviudals(self):
        pass


class TournamentSelection(Selection):
    """
    Clase para representar la selección por torneo en un algoritmo genético.

    Args:
        chromosomes (list): Lista de cromosomas.
        SOLUTIONS (numpy.ndarray): Espacio de soluciones.
        k (int): Tamaño del torneo.
    """

    def __init__(self, chromosomes, k=3):
        self.chromosomes = chromosomes
        self.k = k

    def tournament(self, selected_individuals):
        participants = [chromosome for chromosome in self.chromosomes if chromosome not in selected_individuals]
        participants = random.sample(participants, self.k)
        participants.sort(key=lambda x: x.fitness)
        winner = participants[0]
        return winner

    def get_individuals(self, n_repetition=10):
        selected = []
        for _ in range(n_repetition):
            winner = self.tournament(selected)
            selected.append(winner)
        return selected

In [12]:
NUM_POP = 50#[40, 50, 60, 70, 80, 90, 100]
k_sample = 20#[10, 20, 30, 40]
EPOCHS = 5
criterion = nn.CrossEntropyLoss()


batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [13]:
def genetic_algorithm(train_loader, val_loader, test_loader, model, criterion,\
                      epochs=5, learning_rate=1e-4, image_size = (64, 64), neurons_candidates = [32, 64, 128, 256, 512],\
                      NUM_POP=50, generations=10, k_sample=20, percentage_parents_selection=0.7, crossover_rate=0.5, \
                      percentage_elitism=0.1, mutation_rate=0.5
                      ):
  #init population
  pop = init_pop(NUM_POP, neurons_candidates = neurons_candidates)
  chromosome = build_chromosome(pop, train_loader, val_loader, test_loader, model, criterion,\
                     epochs=epochs, learning_rate=learning_rate, image_size=image_size)

  tournament_selecciton = TournamentSelection(chromosome, k=k_sample)
  history_best_fitness = []

  for g in generations:
      print(f"===Generation: {g}/{generations}===")
      random.shuffle(chromosome)
      new_population = []
      num_parents_candidates = int(len(chromosome)*percentage_parents_selection)
      #parents selection step
      selected_parents = tournament_selecciton.get_indiviudals(num_parents_candidates)

      random.shuffle(selected_parents)
      #crossover step
      for i in range(0, len(selected_parents), 2):
        if i + 1 < len(selected_parents):
            parent1 = selected_parents[i]
            parent2 = selected_parents[i + 1]

            descendant1, descendant2 = crossover_two_points(parent1.chromosome, parent2.chromosome, crossover_rate=crossover_rate)
            new_population.extend([descendant1, descendant2])

      #mutation step
      for i in range(len(new_population)):
        new_population[i] = mutation(new_population[i], mutation_rate)
      new_population = list(np.unique(new_population), axis=0)
      new_chromosomes = build_chromosome(new_population, train_loader, val_loader, test_loader, model, criterion,\
                     epochs=epochs, learning_rate=learning_rate, image_size=image_size)

      #best_fitness
      new_chromosomes.sort(key=lambda x: x.fitness)
      best_fitness= new_chromosomes[0]
      history_best_fitness.append(best_fitness)

      #generational gap
      #elitism
      new_chromosomes = new_chromosomes[:int(len(new_chromosomes) * percentage_elitism)]
      chromosome[:len(new_chromosomes)] = new_chromosomes
      print(f"End generation, best fitness: {best_fitness.fitness} using config: {best_fitness.chromosome}")
  return history_best_fitness

In [14]:
#genetic_algorithm(train_loader, val_loader, test_loader, EvolvedCNN, criterion,\
#                      epochs=5, learning_rate=1e-4, image_size = (64, 64), neurons_candidates = [32, 64, 128, 256, 512],\
#                      NUM_POP=15, generations=10, k_sample=5, percentage_parents_selection=0.7, crossover_rate=0.5, \
#                      percentage_elitism=0.1, mutation_rate=0.5
#                      )

#Tuning Parameters

-  In this phase, we will optimize the hyperparameters of the Genetic Algorithm. We build instances for our Genetic Algorithm and optimize the numeric hyperparameters (such as num_pop, k_sample for tournament, crossover_rate, mutation_rate).

In [15]:
num_vectors_params = 20

def generate_parameter_vector(percentage_elitism_space, percentage_parents_selection_space,
                              k_sample_space, num_pop_space, crossover_rate_space, mutation_rate):
    """
    Generates a parameter vector for a genetic algorithm.

    Args:
        percentage_elitism_space (numpy.ndarray): Solution space for percentage_elitism.
        percentage_parents_selection_space (numpy.ndarray): Solution space for percentage_parents_selection.
        k_sample_space (numpy.ndarray): Solution space for k_sample.
        num_pop_space (numpy.ndarray): Solution space for num_pop.
        crossover_rate_space (numpy.ndarray): Solution space for crossover_rate.

    Returns:
        list: Parameter vector [percentage_elitism, percentage_parents_selection, k_sample, num_pop, crossover_rate].
    """
    while True:
        random_percentage_elitism = np.random.choice(percentage_elitism_space)
        random_percentage_parents_selection = np.random.choice(percentage_parents_selection_space)
        random_k_sample = np.random.choice(k_sample_space)
        random_num_pop = np.random.choice(num_pop_space)
        random_crossover_rate = np.random.choice(crossover_rate_space)
        random_mutation_rate = np.random.choice(mutation_rate)
        if random_k_sample < random_num_pop:
            break

    return [random_percentage_elitism, random_percentage_parents_selection, random_k_sample, random_num_pop, random_crossover_rate, mutation_rate]

def repet_genetic_algorithm(train_loader, val_loader, test_loader, EvolvedCNN, criterion,\
                      epochs=5, learning_rate=1e-4, image_size = (64, 64), neurons_candidates = [32, 64, 128, 256, 512],\
                      NUM_POP=15, generations=10, k_sample=5, percentage_parents_selection=0.7, crossover_rate=0.5, \
                      percentage_elitism=0.1, mutation_rate=0.5, num_repetition=10):

    i = 0
    repet_history_best_fitness = []
    while i < num_repetition:
        history_best_fitness = genetic_algorithm(train_loader, val_loader, test_loader, EvolvedCNN, criterion,\
                            epochs=epochs, learning_rate=learning_rate, image_size = image_size,
                            neurons_candidates = neurons_candidates,\
                            NUM_POP=NUM_POP, generations=generations, k_sample=k_sample,\
                            percentage_parents_selection=percentage_parents_selection,\
                            crossover_rate=crossover_rate, percentage_elitism=percentage_elitism,
                            mutation_rate=mutation_rate
                            )
        repet_history_best_fitness.append(history_best_fitness)
        i+=1
    return repet_history_best_fitness

In [ ]:
percentage_elitism_space = np.linspace(0.05, 0.2, 10)
percentage_parents_selection_space = np.linspace(0.5, 0.9, 10)
crossover_rate_space = np.linspace(0.5, 0.9, 10)
mutation_rate = np.linspace(0.1, 0.9, 10)
k_sample_space = np.arange(10, 40, 5)
num_pop_space = np.arange(30, 80, 10)

vectors_used = []
N = 10
while len(vectors_used) < N:
  vector_parametros = generate_parameter_vector(percentage_elitism_space,
                              percentage_parents_selection_space, k_sample_space, num_pop_space, crossover_rate_space, mutation_rate)

  print("Vector de parámetros:", vector_parametros)
  repet_history_best = repet_genetic_algorithm(train_loader, val_loader, test_loader, EvolvedCNN, criterion,\
                      epochs=5, learning_rate=1e-4, image_size = (64, 64), neurons_candidates = [32, 64, 128, 256, 512],\
                      NUM_POP=vector_parametros[3], generations=10, k_sample=vector_parametros[2],
                      percentage_parents_selection=vector_parametros[1], crossover_rate=vector_parametros[4], \
                      percentage_elitism=vector_parametros[0], mutation_rate=vector_parametros[5], num_repetition=10)
  vectors_used.append((vector_parametros, repet_history_best))